<a href="https://colab.research.google.com/github/Autobot37/gpt.cpp/blob/main/gptcpp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install einops

In [ ]:
import numpy as np
import torch
from einops import rearrange
from torch import nn



dim = 4
to_qvk = nn.Linear(dim, dim * 3, bias=False)

x = torch.randn((4,4))

# Step 1
qkv = to_qvk(x)  # [tokens, dim*3 ]

# decomposition to q,v,k
# rearrange tensor to [3,tokens, dim] and cast to tuple
#q, k, v = tuple(rearrange(qkv, 't (d k) -> k t d ', k=3))
q = [[0]*4 for _ in range(4)]
k = [[0]*4 for _ in range(4)]
v = [[0]*4 for _ in range(4)]

for i in range(4):
  for j in range(4):
    q[i][j] = qkv[i][3*j]
    k[i][j] = qkv[i][3*j+1]
    v[i][j] = qkv[i][3*j+2]

q = torch.tensor(q)
k = torch.tensor(k)
v = torch.tensor(v)


In [ ]:
n_head = 2
head_size = 2
block_size = 4
n_embd = 4

In [ ]:
q.shape,qkv.shape

(torch.Size([4, 4]), torch.Size([4, 12]))

In [ ]:
qkv,k

(tensor([[ 0.3235, -0.2445, -0.5229, -0.4231, -0.1864,  0.5530, -0.1740, -0.9017,
          -0.2983,  0.3473,  0.4393,  0.2487],
         [-0.0996,  0.2137,  0.5813,  0.4434, -0.2762, -0.0313, -0.8811,  0.7465,
          -0.2741, -0.1409, -0.4585,  0.3756],
         [ 0.0424, -0.5876,  0.4334, -0.6915,  1.0802, -0.7285,  0.7560,  0.5750,
          -0.1491, -0.1219,  0.5656, -0.9532],
         [-0.5714,  0.0051, -0.6259,  0.2996, -0.1645,  0.4157,  0.4780, -0.8865,
           0.7962, -0.2889,  0.0950,  0.1860]], grad_fn=<MmBackward0>),
 tensor([[-0.2445, -0.1864, -0.9017,  0.4393],
         [ 0.2137, -0.2762,  0.7465, -0.4585],
         [-0.5876,  1.0802,  0.5750,  0.5656],
         [ 0.0051, -0.1645, -0.8865,  0.0950]]))

In [ ]:
print((q == qkv[:,0::3]).all())
print((k == qkv[:,1::3]).all())
print((v == qkv[:,2::3]).all())

tensor(True)
tensor(True)
tensor(True)


In [ ]:
k

tensor([[-0.2445, -0.1864, -0.9017,  0.4393],
        [ 0.2137, -0.2762,  0.7465, -0.4585],
        [-0.5876,  1.0802,  0.5750,  0.5656],
        [ 0.0051, -0.1645, -0.8865,  0.0950]])

In [ ]:
k = k.view(4, 2, 4 // 2).transpose(0, 1)

In [ ]:
k,k.shape

(tensor([[[-0.2445, -0.1864],
          [ 0.2137, -0.2762],
          [-0.5876,  1.0802],
          [ 0.0051, -0.1645]],
 
         [[-0.9017,  0.4393],
          [ 0.7465, -0.4585],
          [ 0.5750,  0.5656],
          [-0.8865,  0.0950]]]),
 torch.Size([2, 4, 2]))

In [ ]:
qkv#1 4 7 10 13 16

tensor([[ 0.3235, -0.2445, -0.5229, -0.4231, -0.1864,  0.5530, -0.1740, -0.9017,
         -0.2983,  0.3473,  0.4393,  0.2487],
        [-0.0996,  0.2137,  0.5813,  0.4434, -0.2762, -0.0313, -0.8811,  0.7465,
         -0.2741, -0.1409, -0.4585,  0.3756],
        [ 0.0424, -0.5876,  0.4334, -0.6915,  1.0802, -0.7285,  0.7560,  0.5750,
         -0.1491, -0.1219,  0.5656, -0.9532],
        [-0.5714,  0.0051, -0.6259,  0.2996, -0.1645,  0.4157,  0.4780, -0.8865,
          0.7962, -0.2889,  0.0950,  0.1860]], grad_fn=<MmBackward0>)

In [ ]:
compk = k

In [ ]:
kd = torch.zeros_like(k)

#n_head, block_size, head_size

for i in range(n_head):
  for j in range(block_size):
    for k in range(head_size):
      kd[i][j][k] = qkv[j][3*k + 1 + 6*i]


In [ ]:
qkv

tensor([[ 0.3235, -0.2445, -0.5229, -0.4231, -0.1864,  0.5530, -0.1740, -0.9017,
         -0.2983,  0.3473,  0.4393,  0.2487],
        [-0.0996,  0.2137,  0.5813,  0.4434, -0.2762, -0.0313, -0.8811,  0.7465,
         -0.2741, -0.1409, -0.4585,  0.3756],
        [ 0.0424, -0.5876,  0.4334, -0.6915,  1.0802, -0.7285,  0.7560,  0.5750,
         -0.1491, -0.1219,  0.5656, -0.9532],
        [-0.5714,  0.0051, -0.6259,  0.2996, -0.1645,  0.4157,  0.4780, -0.8865,
          0.7962, -0.2889,  0.0950,  0.1860]], grad_fn=<MmBackward0>)

In [ ]:
compk.transpose(-2,-1)

tensor([[[-0.2445,  0.2137, -0.5876,  0.0051],
         [-0.1864, -0.2762,  1.0802, -0.1645]],

        [[-0.9017,  0.7465,  0.5750, -0.8865],
         [ 0.4393, -0.4585,  0.5656,  0.0950]]])

In [ ]:
origtk = compk.transpose(-2,-1)
origtk.shape

torch.Size([2, 2, 4])

In [ ]:
tk = torch.zeros_like(origtk)
for i in range(n_head):
  for j in range(head_size):
    for k in range(block_size):
      tk[i][j][k] = qkv[k][3*j + 6*i + 1]


In [ ]:
origtk.shape,tk.shape

(torch.Size([2, 2, 4]), torch.Size([2, 2, 4]))

In [ ]:
torch.allclose(origtk,tk)

True

In [ ]:
compk

tensor([[[-0.2445, -0.1864],
         [ 0.2137, -0.2762],
         [-0.5876,  1.0802],
         [ 0.0051, -0.1645]],

        [[-0.9017,  0.4393],
         [ 0.7465, -0.4585],
         [ 0.5750,  0.5656],
         [-0.8865,  0.0950]]])

In [ ]:
torch.allclose(compk,kd)

True

In [ ]:
torch.allclose(kd,compk)

True

In [ ]:
#simulating 3d multiplication without distrbing 0th dim
a = torch.randn((2,4,6))
b = torch.randn((2,6,4))

In [ ]:
tc = a@b

In [ ]:
c = torch.zeros((2,4,4))

In [ ]:
shape_a = (2, 4, 6)
shape_b = (2, 6, 4)
shape_c = (2, 4, 4)
for i in range(shape_a[0]):  # Iterate over the first dimension (2)
    for j in range(shape_a[1]):  # Iterate over the second dimension (4)
        for k in range(shape_b[2]):  # Iterate over the third dimension (4)
            for m in range(shape_a[2]):  # Iterate over the inner dimension (6)
                c[i, j, k] += a[i, j, m] * b[i, m, k]

In [ ]:
torch.allclose(tc,c)

True

In [ ]:
c.shape

torch.Size([2, 4, 4])

In [ ]:
a = torch.randn((2,2,2))
torch.nn.functional.softmax(a,dim=-1)

tensor([[[0.7354, 0.2646],
         [0.4193, 0.5807]],

        [[0.8538, 0.1462],
         [0.2116, 0.7884]]])

In [ ]:
soft = torch.nn.functional.softmax(c,dim=-1)
att_softmax = torch.zeros_like(c)
att = c

In [ ]:
flat = att.flatten().tolist()


In [ ]:
for i in range(n_head):
    for j in range(block_size):
        row_max = max(att[i, j, :])
        row_sum = 0.0
        for k in range(block_size):
            att[i, j, k] = np.exp(att[i, j, k] - row_max)
            row_sum += att[i, j, k]
        for k in range(block_size):
            att[i, j, k] /= row_sum

In [ ]:
torch.allclose(att,soft)

False

In [ ]:
n_head = 2
head_size = 2
block_size = 4
n_embd = 4

In [ ]:
k3d = [[[0]*2 for _ in range(4)] for _ in range(2)]
k3d = torch.tensor(k3d,dtype=torch.float32)
k3d.shape

torch.Size([2, 4, 2])

In [ ]:
for i in range(n_head):
  for b in range(block_size):
    for h in range(head_size):
      k3d[i][b][h] = k[b][i*n_head + h]

In [ ]:
(k3d == k).all()

In [ ]:
torch.tril(torch.ones(block_size,block_size)).view(1, 1, block_size, block_size)

tensor([[[[1., 0., 0., 0.],
          [1., 1., 0., 0.],
          [1., 1., 1., 0.],
          [1., 1., 1., 1.]]]])

In [ ]:
knew = [[0]*4 for _ in range(4)]
knew = torch.tensor(knew,dtype=torch.float32)

In [ ]:
for h in range(n_head):
    for b in range(block_size):
        for hs in range(head_size):
            knew[b][h * head_size + hs] = qkv[b][3 * hs  + 3*((h!=0)*h+1) + 1]


In [ ]:
print((k[0] == knew[:, :2]).all())
(k[1] == knew[:,2:4]).all()

In [ ]:
k.shape,knew.shape

(torch.Size([4, 4]), torch.Size([4, 4]))

In [ ]:
k

In [ ]:
knew

In [ ]:
#############mha in parallel

In [ ]:
import torch
import torch.nn as nn

In [ ]:
torch.manual_seed(1087)

qkv = nn.Linear(n_embd,3*n_embd)




In [ ]:
x = torch.randn((block_size,n_embd))

In [ ]:
qkv(x)

tensor([[ 0.9931,  0.4669,  0.0328, -0.7106, -0.4818,  0.8863, -0.7688, -0.1780,
          0.9150,  1.0797,  0.0990, -0.9394],
        [-0.4884, -0.4681,  0.2540,  0.5969, -0.1577,  0.2435,  0.3056, -0.5626,
          0.3166,  0.3921, -0.4592, -0.1502],
        [-0.5591, -0.2962,  0.2702,  0.3848, -0.0114,  0.1145,  0.1177, -0.5324,
          0.5960,  0.3783, -0.4841, -0.1353],
        [-0.8677, -1.1613, -0.3105,  0.8271,  0.2495,  0.2890,  1.0380, -0.1065,
         -0.1906,  0.4826, -0.8630,  1.0692]], grad_fn=<AddmmBackward0>)

In [ ]:
q,k,v = qkv(x).split(n_embd,dim=1)
q

tensor([[ 0.9931,  0.4669,  0.0328, -0.7106],
        [-0.4884, -0.4681,  0.2540,  0.5969],
        [-0.5591, -0.2962,  0.2702,  0.3848],
        [-0.8677, -1.1613, -0.3105,  0.8271]], grad_fn=<SplitBackward0>)

In [ ]:
qkv(x)[:, :n_embd]

tensor([[ 0.9931,  0.4669,  0.0328, -0.7106],
        [-0.4884, -0.4681,  0.2540,  0.5969],
        [-0.5591, -0.2962,  0.2702,  0.3848],
        [-0.8677, -1.1613, -0.3105,  0.8271]], grad_fn=<SliceBackward0>)

In [ ]:
#qk,v = shape == block_size, n_embd means vocab_size, that when softmaxed will produce next token anytime

In [ ]:
qv = q.view(block_size, n_head,n_embd//n_head).transpose(0,1)
kv = k.view(block_size, n_head,n_embd//n_head).transpose(0,1)
vv = v.view(block_size, n_head,n_embd//n_head).transpose(0,1)


In [ ]:
qv.shape#for two head 4 dims , 4  + 4 = 8 = N_EMBD


torch.Size([2, 4, 2])

In [ ]:
att = qv@kv.transpose(1,2)
att[0].shape

torch.Size([4, 4])

In [ ]:
#or
qv[0] == q[:,:n_embd//n_head]

tensor([[True, True],
        [True, True],
        [True, True],
        [True, True]])

In [ ]:
for i in range(n_head):
  mq = q[:,i*head_size:(i+1)*head_size]
  mk = k[:,i*head_size:(i+1)*head_size]
  print((mq==qv[0]).all())
  print((mk==kv[0]).all())
  matt = mq@(mk.transpose(0,1))
  print(torch.allclose(matt, att[0]))
  break



tensor(True)
tensor(True)
True


In [ ]:
bias = torch.tril(torch.ones(block_size,block_size).view(1,block_size,block_size))

In [ ]:
bias

tensor([[[1., 0., 0., 0.],
         [1., 1., 0., 0.],
         [1., 1., 1., 0.],
         [1., 1., 1., 1.]]])

In [ ]:
T = block_size
attm = att.masked_fill(bias[:,:T,:T] == 0, float('-inf'))

In [ ]:
att = att @ bias

In [ ]:
for i in range(n_head):
  for j in range(block_size):
    for k in range(block_size):
      if bias[0][j][k] == 0:
        att[i][j][k] = float('-inf')

In [ ]:
att,attm

(tensor([[[ 0.5574,    -inf,    -inf,    -inf],
          [-0.8893, -1.0496,    -inf,    -inf],
          [-0.8621, -1.0149, -0.3741,    -inf],
          [-0.5085, -0.4864, -0.3110, -0.0826]],
 
         [[-0.3181,    -inf,    -inf,    -inf],
          [-0.0768,  0.3728,    -inf,    -inf],
          [-1.2234, -2.3084, -0.7263,    -inf],
          [-0.0960,  0.5384, -0.2103,  0.5963]]]),
 tensor([[[-0.2200,    -inf,    -inf,    -inf],
          [ 0.1604, -0.6660,    -inf,    -inf],
          [ 0.1528, -0.6408,  0.2757,    -inf],
          [-0.0222, -0.1754, -0.2284, -0.0826]],
 
         [[ 0.1368,    -inf,    -inf,    -inf],
          [-0.4496,  0.5288,    -inf,    -inf],
          [ 1.0850, -1.5821,  0.5863,    -inf],
          [-0.6344,  0.7487, -0.8066,  0.5963]]]))

In [ ]:
#att = attm = n_head, block_size, block_size

In [ ]:
yl = attm @ v# is n_head, bsz. head_size

In [ ]:
y = yl
yl = yl.transpose(0, 1).contiguous()

In [ ]:
ff = []

In [ ]:
for i in range(block_size):
  fl = yl[i].flatten().tolist()
  print(fl)
  ff.append(fl)


[inf, -inf, nan, inf, inf, -inf, nan, inf]
[inf, -inf, nan, inf, inf, -inf, nan, inf]
[inf, -inf, -inf, inf, inf, -inf, -inf, inf]
[0.406660795211792, -0.4270693063735962, 0.13125061988830566, 0.36470210552215576, 1.5916489362716675, 0.3409857749938965, -0.2002646028995514, -0.29666003584861755]


In [ ]:
x = yl.flatten().tolist()

In [ ]:
x[1*n_embd+3] == ff[1][3]

True

In [ ]:
#now tokenizer bro

In [ ]:
from typing import List

In [ ]:
vocab = {
    'l o w </w>': 5,
    'l o w e r </w>': 2,
    'n e w e s t </w>': 6,
    'w i d e s t </w>': 3,
    'h a p p i e r </w>': 2
}

In [ ]:
def get_pair(vocab):
  pairs = {}
  for word, freq in vocab.items():
    sym = word.split()

    for i in range(len(sym)-1):
      pair = (sym[i],sym[i+1])
      pairs[pair] = pairs.get(pair,0)+freq
  return pairs

In [ ]:
pairs = get_pair(vocab)

In [ ]:
import re
best_pair = max(pairs,key=pairs.get)
patt = re.escape(' '.join(best_pair))
repl = ''.join(best_pair)
vocab_out = {}
for word in vocab:
  out = re.sub(patt,repl,word)
  vocab_out[out] = vocab[word]


In [ ]:
vocab_out

{'l o w </w>': 5,
 'l o w e r </w>': 2,
 'n e w es t </w>': 6,
 'w i d es t </w>': 3,
 'h a p p i e r </w>': 2}